In [ ]:
# PSEUDOCODE
# 1. implement an ASR algorithm which can take in the audio file and return an transcript of the conversation
#   a Devise a test in order to ensure that the model is working correctly
# 2. implement BART transformer (or other architecture) for the purpose of summarization of patient-clinician interactions
#   a. Use ROUGE score(s) to benchmark model performance while training
#   b. Use MEDCON (from ACI-Bench paper) to do full model benchmarking
#   c. Can also use other benchmarks, like HELM, to do this
#   d. See 'Model Training and Benchmarking' for more information.



In [2]:
# 2. Implement LLM to summarize patient-clinician conversation.
!pip install datasets
!pip install accelerate -U
!pip install rouge.score nltk py7zr



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.7 M

In [7]:
!pip install transformers[torch]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# get ACI-Bench dataset
# NOTE run this line at least once
!git clone https://github.com/wyim/aci-bench.git

Mounted at /content/drive
Cloning into 'aci-bench'...
remote: Enumerating objects: 528, done.
remote: Counting objects: 100% (528/528), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 528 (delta 357), reused 488 (delta 328), pack-reused 0
Receiving objects: 100% (528/528), 8.98 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [9]:
import transformers
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import time
# import evaluate
import pandas as pd
import numpy as np
import os

In [5]:
from datasets import load_dataset, load_from_disk,load_metric

csv_asr_test1_data_aci = '/content/aci-bench/data/src_experiment_data/test1_aci_asr.csv'

csv_asrcorr_train_data = '/content/aci-bench/data/src_experiment_data/train_aci_asrcorr.csv'

csv_asr_validate = '/content/aci-bench/data/src_experiment_data/valid_aci_asr.csv'

dataset = load_dataset('csv', data_files={'train':[csv_asrcorr_train_data], 'test':[csv_asr_test1_data_aci], 'validate':[csv_asr_validate]})
print(dataset)
# dataset = load_dataset('csv',data_files={'train':[csv_virtasr_train_data,csv_asrcorr_train_data, train_data], 'test':[csv_asr_test1_data_aci ,csv_asr_test2_data_aci, csv_asr_test3_data_aci, csv_virtasr_test1_data, csv_virtasr_test2_data, csv_virtasr_test3_data, csv_virthuman_test1_data, csv_virthuman_test2_data, csv_virthuman_test3_data, csv_asrcorr_test1_data_aci, csv_asrcorr_test2_data_aci, csv_asrcorr_test3_data_aci, clinicalnlp_taskB_test1, clinicalnlp_taskC_test2], 'validate': [csv_asr_validate,csv_asrcorr_validate,csv_virtasr_validate,csv_virthuman_validate,valid_data]},column_names=['dataset','id','dialogue','note'])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validate split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'dialogue', 'note'],
        num_rows: 35
    })
    test: Dataset({
        features: ['dataset', 'id', 'dialogue', 'note'],
        num_rows: 22
    })
    validate: Dataset({
        features: ['dataset', 'id', 'dialogue', 'note'],
        num_rows: 11
    })
})


In [10]:
model_checkpoints = "facebook/bart-large-xsum"

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
# original_model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=original_model)
metric = load_metric('rouge')

<ipython-input-10-e6e55b095ba4>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 406290432
all model parameters: 406290432
percentage of trainable model parameters: 100.00%


In [12]:
max_input = 1024
max_target = 128
batch_size = 3

def preprocess_data(data_to_process):
  """"""
  print('Preprocessing data')
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length',truncation=True)
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['note'],max_length=max_target, padding='max_length',truncation=True)
  # set labels
  model_inputs['labels'] = targets['input_ids']
  # return the tokenized data
  return model_inputs

tokenize_data = dataset.map(preprocess_data, batched = True)


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Preprocessing data


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Preprocessing data


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Preprocessing data


In [13]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
# define parameters for dataset
args = Seq2SeqTrainingArguments(
    'medical_note_hpi', #
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=False  #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    original_model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validate'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics= compute_rouge
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [1]:
index = 20
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['note']

# prompt = f"""
# {dialogue}"""
validate = """[doctor] hey diana it's good to see you in here so i see that you injured your wrist could you tell me a bit about what happened [patient] yeah i was walking up and down the stairs i was doing my laundry and i slipped and i tried to catch myself and i put my arms out to catch myself and then all of a sudden i just my wrist started to hurt real bad and it got real swollen [doctor] wow okay so which wrist are we talking about left or right [patient] it's my right one of course [doctor] okay and then have you ever injured this arm before [patient] no i have not [doctor] okay alright so on a scale of one to ten how severe is the pain [patient] gosh it's like a nine [doctor] wow okay have you done anything to ease it [patient] yeah i did the ice thing i put ice on it and then i you know i even i have a ace wrap at home i try to do that [doctor] mm-hmm [patient] and then i took some ibuprofen but it helps a little bit but it's just it's it's just not right [doctor] okay [patient] really [doctor] yeah okay have you sorry i'm trying to think how long ago did this injury happen [patient] this happened yesterday morning [doctor] okay [patient] maybe just you know i just bumped it but [doctor] okay [patient] it's just not it's really bad [doctor] okay no i understand okay so i'm going so you said you were doing laundry [patient] yes i had my back hit my basket and for some reason this cold started to kinda fall out a little bit i was trying to catch it i missed a step and i just totally [doctor] okay alright any does the pain extend anywhere [patient] no not really [doctor] okay [patient] it's just really along my wrist [doctor] okay any numbness any tingling [patient] a little one and one ca n't tell if it's just because of the swelling in my wrist but just i can like i can feel it my fingers still [doctor] mm-hmm [patient] but just maybe a little bit of tingling [doctor] okay alright and are you so so okay i'm gon na think on this but in the meantime i'm gon na do my physical exam alright [patient] okay [doctor] okay so you know looking at your looking at your head and your neck i do n't appreciate any like adenopathy no thyromegaly no no carotid bruit looking at your listening to your heart i do n't appreciate any murmur no rub no gallop your lungs are clear to auscultation bilaterally your lower legs you have palpable pulses no lower edema your shoulders every like your upper extremities i see normal range of movement with your right wrist let's go ahead and focus on it so when i push on the inside here does it hurt [patient] yes [doctor] okay [patient] it does [doctor] and what about the outside does that hurt as well [patient] yeah it does [doctor] are you able to move your wrist towards your arm like [patient] not without extreme pain [doctor] okay so pain on flexion what about extension when you pick your wrist up [patient] yeah i have a hard time doing that actually [doctor] alright what about we're gon na go ahead and hold your arm like straight like flat and then try and move it sideways does radial deviation hurt [patient] yeah [doctor] alright and then lateral as well [patient] yeah it's really hard to move any direction of this hand for some reason [doctor] alright so wrist abduction adduction positive for pain on movement are you able to make a fist [patient] hmmm yeah a little bit but i ca n't do it really tight [doctor] okay alright okay so i'm just gon na go ahead and feel on your fingers really quickly alright metacarpals intact noticed some obvious swelling ecchymosis obvious swelling and bruising tenderness on palpation throughout there is evidence of potential fracture feeling some bony crepitus alright so this pain is it like chronic i wanted to ask you [patient] yeah i would say it kinda goes away when i take that ibuprofen but for the most part i feel it i feel it there and it it's just really really bad when i move it all [doctor] okay so when you like is there a position either hurts less or hurts more like say if your arm is raised and elevated over your head does it hurt more or is it just best to keep it like down [patient] it's good if i keep it a little bit above my like a little i guess a little bit like around my like just a regular level like if you're typing or something and then i just put it on a pillow and i just let it stay straight like i feel better [doctor] okay yeah no i do n't think i understand completely okay so i took a look at your vitals and your blood pressure is a little elevated but honestly that's probably to do with the pain right our body can respond to pain in that way we are looking at like a hundred and forty over over seventy it's not anything crazy but something to mention i see that your heart rate is also a little elevated at like about like eighty beats a minute you are not running a fever so that's great look at ninety ninety seven . two your respiratory rate is pretty normal at like twenty so before we came in i i know that we had you do an x-ray and i'm sure that that was a bit more painful because we had to do so many manipulations but i do wan na note that you are positive for what we call a colles' fracture what that means is that the joints between your wrist like the bones between your wrist that there there is evidence of a a fracture and we are gon na have to treat it a little conservatively at first and then consider some of the options options that are available to us so for your primary diagnosis of a colles' fracture we are going to give you a thumb spica for today and that's going to [patient] i'm sorry [doctor] pardon what [patient] a what [doctor] we're gon na brace you we're gon na give you a brace [patient] okay thank you [doctor] sorry no problem sorry yeah not a thumb spica we're gon na brace your arm and you're gon na have that we we have a couple of options but i think the best course of action is gon na be for surgery we will in the meantime give you pain medication i wan na put you on fifty milligrams of ultram every six hours and then i also wan na get you on get you into physical therapy a few weeks after surgery this is gon na be just a normal procedure you will be in for an overnight stay but after that once we assess and make sure that everything is good you'll be able to go home okay [patient] when do i have to have the surgery [doctor] we would like it to happen as quickly as possible you know your body is a wonderful miracle and it's going to start trying to heal on it's own what we need to do is get your wrist straight and then like put screws in to make sure that we hold it in place or else it could like heal and malform [patient] okay [doctor] alright so what [patient] how how long do i have to wear that brace [doctor] you're gon na be wearing the brace for about six weeks [patient] six weeks [doctor] yeah so you're gon na you're gon na come in for your surgery we're gon na perform it you're gon na stay overnight and then you'll be bracing it for six weeks in the meantime you'll also then go to physical therapy i want you there like we're gon na they're gon na do an assessment and determine how much but i'm thinking probably three times a week just to make sure that you can get your wrist as strong as possible to prevent like future injury now the cool thing about getting any kind of a bone break is that your your body comes out even stronger so this should n't happen again but unfortunately like it's these situations that oof that just kind of [patient] oof [doctor] these these deform these deformities that really that really kind of hurt is the short version alright no problem any other questions [patient] no well i am going on vacation do i need to cancel it like can i still go even with the i mean after the surgery [doctor] yeah [patient] do it as soon as possible i'm going a vacation in a month so [doctor] okay how long is the vacation [patient] it's only for like a couple weeks [doctor] okay well so you might have to postpone it just because depending on what physical therapy says right if they feel that you can sustain if you can like sustain the exercises while you're gone that if there's something that you can do by yourself then you should be fine but we do wan na give it you said that it's gon na happen in a couple of weeks [patient] no vacation in a month [doctor] okay okay yeah so how about in a month we come you come back let's do a checkup again see where we are at and then we can assess whether or not this is something that i would recommend you do [patient] that sounds good thank you [doctor] no problem bye [patient] bye [doctor] the fracture appears extra-articular and usually proximal to the radial ulnar joint dorsal angulation of the distal fracture fragment is present to a variable degree if dorsal angulation is severe presenting with a dinner fork deformity ulnar styloid fracture is present"""

input_ids = tokenizer(validate, return_tensors="pt").input_ids
model_inputs = tokenizer(validate,  max_length=max_input, padding='max_length', truncation=True)

inputs = tokenizer(validate, max_length=max_input,padding='max_length',truncation=True)
# original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=5))
# original_model_outputs = original_model.generate(inputs.input_ids, num_beams=5, min_length=50, max_length=100)
raw_pred, _, _ = trainer.predict([model_inputs])

# original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
# output = tokenizer.decode(
#     original_model.generate(
#         inputs["input_ids"],
#         max_new_tokens=200,
#     )[0],
#     skip_special_tokens=True
# )

# dash_line = '-'.join('' for x in range(100))
# print(dash_line)
# print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
# print(dash_line)
# print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

NameError: name 'dataset' is not defined